# TensorFlow实现VGGNet-16

In [1]:
# -*- coding: utf-8 -*-
# author: JiRuiqing
# email: ruiqing0706@gmail.com

# 导包
from datetime import datetime
import math
import time
import tensorflow as tf

In [2]:
# 设定batch大小和数量
batch_size = 32
num_batches = 100

In [3]:
# 创建卷积层并把本层的参数存入参数列表
def conv_op(input_op, name, kh, kw, n_out, dh, dw, p):
    n_in = input_op.get_shape()[-1].value
    
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(scope+"w", shape=[kh, kw, n_in, n_out], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer_conv2d())
        
        conv = tf.nn.conv2d(input_op, kernel, (1, dh, dw, 1), padding='SAME')
        bias_init_val = tf.constant(0.0, shape=[n_out], dtype=tf.float32)
        biases = tf.Variable(bias_init_val, trainable=True, name='b')
        z = tf.nn.bias_add(conv, biases)
        activation = tf.nn.relu(z, name=scope)
        p += [kernel, biases]
        
        return activation

In [4]:
# 创建全连接层并把本层的参数存入参数列表
def fc_op(input_op, name, n_out, p):
    n_in = input_op.get_shape()[-1].value
    
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(scope+"w", shape=[n_in, n_out], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.Variable(tf.constant(0.1, shape=[n_out], dtype=tf.float32), name='b')
        activation = tf.nn.relu_layer(input_op, kernel, biases, name=scope)
        p += [kernel, biases]
        return activation

In [5]:
# 创建最大池化层
def mpool_op(input_op, name, kh, kw, dh, dw):
    return tf.nn.max_pool(input_op, ksize=[1, kh, kw, 1], strides=[1, dh,dw, 1], padding='SAME', name=name)

In [6]:
# 创建VGGNet-16的网络结构
# 6部分，前5段为卷积网络，最后一段是全连接网络
def inference_op(input_op, keep_prob):
    
    p = []
    
    # 第一段卷积网络（两个卷积层+一个最大池化层）
    conv1_1 = conv_op(input_op, name="conv1_1", kh=3, kw=3, n_out=64, dh=1, dw=1, p=p)
    
    conv1_2 = conv_op(conv1_1, name="conv1_2", kh=3, kw=3, n_out=64, dh=1, dw=1, p=p)
    
    pool1 = mpool_op(conv1_2, name="pool1", kh=2, kw=2, dw=2, dh=2)
    
    # 第二段卷积网络（两个卷积层+一个最大池化层）
    conv2_1 = conv_op(pool1, name="conv2_1", kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)
    
    conv2_2 = conv_op(conv2_1, name="conv2_2", kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)
    
    pool2 = mpool_op(conv2_2, name="pool2", kh=2, kw=2, dw=2, dh=2)
    
    # 第三段卷积网络（三个卷积层+一个最大池化层）
    conv3_1 = conv_op(pool2, name="conv3_1", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
    
    conv3_2 = conv_op(conv3_1, name="conv3_2", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
    
    conv3_3 = conv_op(conv3_2, name="conv3_3", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
    
    pool3 = mpool_op(conv3_3, name="pool3", kh=2, kw=2, dw=2, dh=2)
    
    # 第四段卷积网络（三个卷积层+一个最大池化层）
    conv4_1 = conv_op(pool3, name="conv4_1", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    
    conv4_2 = conv_op(conv4_1, name="conv4_2", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    
    conv4_3 = conv_op(conv4_2, name="conv4_3", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    
    pool4 = mpool_op(conv4_3, name="pool4", kh=2, kw=2, dw=2, dh=2)
    
    # 第五段卷积网络（三个卷积层+一个最大池化层）
    conv5_1 = conv_op(pool4, name="conv5_1", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    
    conv5_2 = conv_op(conv5_1, name="conv5_2", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    
    conv5_3 = conv_op(conv5_2, name="conv5_3", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    
    pool5 = mpool_op(conv5_3, name="pool5", kh=2, kw=2, dw=2, dh=2)
    
    # 扁平化
    shp = pool5.get_shape()
    flattend_shape = shp[1].value * shp[2].value * shp[3].value
    resh1 = tf.reshape(pool5, [-1, flattend_shape], name="resh1")
    
    # 全连接层
    fc6 = fc_op(resh1, name="fc6", n_out=4096, p=p)
    fc6_drop = tf.nn.dropout(fc6, keep_prob, name="fc6_drop")
    
    # 全连接层
    fc7 = fc_op(fc6_drop, name="fc7", n_out=4096, p=p)
    fc7_drop = tf.nn.dropout(fc7, keep_prob, name="fc7_drop")
    
    # 全连接层
    fc8 = fc_op(fc7_drop, name="fc8", n_out=1000, p=p)
    softmax = tf.nn.softmax(fc8)
    predictions = tf.argmax(softmax, 1)
    return predictions, softmax, fc8, p

In [7]:
def time_tensorflow_run(session, target, feed, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target, feed_dict=feed)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s: step %d, duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' % (datetime.now(), info_string, num_batches, mn, sd))

In [8]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(tf.random_normal([batch_size, image_size, image_size, 3], dtype=tf.float32, stddev=1e-1))
        
        keep_prob = tf.placeholder(tf.float32)
        predictitons, softmax, fc8, p = inference_op(images, keep_prob)

        init = tf.global_variables_initializer()
        session = tf.Session()
        session.run(init)

        time_tensorflow_run(session, predictitons, {keep_prob: 1.0}, "Forward")
        objective = tf.nn.l2_loss(fc8)
        grad = tf.gradients(objective, p)
        time_tensorflow_run(session, grad, {keep_prob: 0.5}, "Forward-backward")

In [9]:
run_benchmark()

2018-01-19 16:28:14.398589: step 0, duration = 14.021
2018-01-19 16:30:32.277283: step 10, duration = 13.387
2018-01-19 16:32:49.219888: step 20, duration = 13.161
2018-01-19 16:35:04.952831: step 30, duration = 13.556
2018-01-19 16:37:23.795569: step 40, duration = 13.383
2018-01-19 16:39:40.281704: step 50, duration = 13.352
2018-01-19 16:41:56.979544: step 60, duration = 13.246
2018-01-19 16:44:14.371298: step 70, duration = 13.211
2018-01-19 16:46:31.384928: step 80, duration = 13.174
2018-01-19 16:48:51.296200: step 90, duration = 12.871
2018-01-19 16:50:42.404611: Forward across 100 steps, 13.620 +/- 0.694 sec / batch
2018-01-19 16:58:11.195086: step 0, duration = 40.840
2018-01-19 17:05:02.237899: step 10, duration = 40.647
2018-01-19 17:11:56.171011: step 20, duration = 40.356
2018-01-19 17:18:42.450428: step 30, duration = 39.721
2018-01-19 17:25:32.224042: step 40, duration = 46.004
2018-01-19 17:32:48.125232: step 50, duration = 41.379
2018-01-19 17:39:42.515656: step 60, du